In [1]:
import json
import argparse
from itertools import chain
from functools import partial
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
import evaluate
from datasets import Dataset, features
import numpy as np
import pandas as pd
import sentencepiece
import sys
sys.path.append('../')
from utils.data_handler import *

/home/kd/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/kd/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/kd/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
DATA_PATH = '/home/kd/Documents/pii_detection/data/'
OUTPUT_DIR = '/home/kd/Documents/pii_detection/models/'
TRAINING_MODEL_PATH = "microsoft/deberta-v3-large"
TRAINING_MAX_LENGTH = 1024

In [3]:
data = do_load_data(DATA_PATH)
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

target = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 
    'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 
    'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL'
]

tokenizer = AutoTokenizer.from_pretrained(TRAINING_MODEL_PATH)

ds = do_hf_dataset(tokenizer, label2id, data, TRAINING_MAX_LENGTH=TRAINING_MAX_LENGTH)

original datapoints:  6807
external datapoints:  4434
mixtral  2355
combined:  9688


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/kd/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map (num_proc=3):   0%|          | 0/9688 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

In [4]:
def tokenize(example, tokenizer, label2id, max_length):

    # rebuild text from tokens
    text = []
    labels = []

    for t, l, ws in zip(
        example["tokens"], example["provided_labels"], example["trailing_whitespace"]
    ):
        text.append(t)
        labels.extend([l] * len(t))

        if ws:
            text.append(" ")
            labels.append("O")

    # actual tokenization
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, max_length=max_length)

    labels = np.array(labels)

    text = "".join(text)
    token_labels = []

    for start_idx, end_idx in tokenized.offset_mapping:
        # CLS token
        if start_idx == 0 and end_idx == 0:
            token_labels.append(label2id["O"])
            continue

        # case when token starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1

        token_labels.append(label2id[labels[start_idx]])

    length = len(tokenized.input_ids)

    return {**tokenized, "labels": token_labels, "length": length}

In [5]:
ds

Dataset({
    features: ['full_text', 'document', 'tokens', 'trailing_whitespace', 'provided_labels', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'labels', 'length'],
    num_rows: 9688
})

In [6]:
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

def compute_metrics(p, all_labels):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

In [7]:
model = AutoModelForTokenClassification.from_pretrained(
    TRAINING_MODEL_PATH,
    num_labels=len(all_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# I actually chose to not use any validation set. This is only for the model I use for submission.
args = TrainingArguments(
    output_dir=OUTPUT_DIR, 
    fp16=True,
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    report_to=["tensorboard"],
    evaluation_strategy="no",
    do_eval=False,
    save_total_limit=1,
    logging_dir='/home/kd/Documents/pii_detection/logs/',
    logging_steps=20,
    lr_scheduler_type='cosine',
    metric_for_best_model="f1",
    greater_is_better=True,
    warmup_ratio=0.1,
    weight_decay=0.01
)

trainer = Trainer(
    model=model, 
    args=args, 
    train_dataset=ds,
    data_collator=collator, 
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, all_labels=all_labels),
)

In [9]:
%%time
trainer.train()

  0%|          | 0/14532 [00:00<?, ?it/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 2.2044, 'learning_rate': 2.2008253094910594e-07, 'epoch': 0.0}
{'loss': 2.1504, 'learning_rate': 4.951856946354883e-07, 'epoch': 0.01}
{'loss': 1.7236, 'learning_rate': 7.702888583218708e-07, 'epoch': 0.01}
{'loss': 1.0526, 'learning_rate': 1.0453920220082531e-06, 'epoch': 0.02}
{'loss': 0.4039, 'learning_rate': 1.3204951856946355e-06, 'epoch': 0.02}
{'loss': 0.2232, 'learning_rate': 1.595598349381018e-06, 'epoch': 0.02}
{'loss': 0.1771, 'learning_rate': 1.8707015130674006e-06, 'epoch': 0.03}
{'loss': 0.1334, 'learning_rate': 2.145804676753783e-06, 'epoch': 0.03}
{'loss': 0.1336, 'learning_rate': 2.4209078404401654e-06, 'epoch': 0.04}
{'loss': 0.0827, 'learning_rate': 2.6960110041265474e-06, 'epoch': 0.04}
{'loss': 0.067, 'learning_rate': 2.97111416781293e-06, 'epoch': 0.05}
{'loss': 0.0756, 'learning_rate': 3.246217331499312e-06, 'epoch': 0.05}
{'loss': 0.0473, 'learning_rate': 3.521320495185695e-06, 'epoch': 0.05}
{'loss': 0.0465, 'learning_rate': 3.796423658872077e-06, 'epo

KeyboardInterrupt: 

In [ ]:
trainer.save_model(OUTPUT_DIR + "/deberta_large_mixtral_3_epochs")
tokenizer.save_pretrained(OUTPUT_DIR + "/deberta_large_mixtral_3_epochs")

('/home/kd/Documents/pii_detection/models//deberta_large_mixtral_4_epochs/tokenizer_config.json',
 '/home/kd/Documents/pii_detection/models//deberta_large_mixtral_4_epochs/special_tokens_map.json',
 '/home/kd/Documents/pii_detection/models//deberta_large_mixtral_4_epochs/spm.model',
 '/home/kd/Documents/pii_detection/models//deberta_large_mixtral_4_epochs/added_tokens.json',
 '/home/kd/Documents/pii_detection/models//deberta_large_mixtral_4_epochs/tokenizer.json')

In [ ]:
torch.cuda.empty_cache()